In [495]:
import pandas as pd
import numpy as np
import itertools
from collections import OrderedDict

In [496]:
def calculate_two_digits(digits, allow_combine=True):
    possible_values_signs = {}

    possible_values_signs[digits[0] + digits[1]] = 'plus'
    possible_values_signs[digits[0] - digits[1]] = 'minus12'
    possible_values_signs[digits[1] - digits[0]] = 'minus21'
    possible_values_signs[digits[0] * digits[1]] = 'multiply'
    if digits[1] != 0:
        possible_values_signs[digits[0] / digits[1]] = 'divide12'
    if digits[0] != 0:
        possible_values_signs[digits[1] / digits[0]] = 'divide21'
    
    if digits[0] < 10 and digits[1] < 10 and type(digits[0]) == int and type(digits[1]) == int and allow_combine:
        if digits[0] != 0:
            possible_values_signs[digits[0] * 10 + digits[1]] = 'combine12'
        if digits[1] != 0:
            possible_values_signs[digits[1] * 10 + digits[0]] = 'combine21'

    if (digits[0] != 0 or digits[1] > 0) and type(digits[1]) == int:
        if digits[0] ** digits[1] < 250:
            possible_values_signs[digits[0] ** digits[1]] = 'factor12'
    if (digits[1] != 0 or digits[0] > 0) and type(digits[0]) == int:
        if digits[1] ** digits[0] < 250:
            possible_values_signs[digits[1] ** digits[0]] = 'factor21'

    return possible_values_signs

In [497]:
def two_two_digits(digits, c):
    digits_copy = digits.copy()
    for i in c:
        try:
            digits_copy.remove(i)
        except ValueError:
            pass
    remaining_c = digits_copy
    return remaining_c

In [498]:
def transform_eq(num1, sign1, num2, sign2, num3, sign3, num4, mode):

    dct = {
        'plus': ' + ',
        'minus': ' - ',
        'multiply': ' * ',
        'divide': ' / ',
        'combine': '',
        'factor': ' ^ '
    }

    # Prep
    if sign1[-2:] == '12' or sign1[-2:] == '21':
        sign1_order = sign1[-2:]
        sign1 = sign1[:-2]
    else:
        sign1_order = '12'

    if sign2[-2:] == '12' or sign2[-2:] == '21':
        sign2_order = sign2[-2:]
        sign2 = sign2[:-2]
    else:
        sign2_order = '12'

    if sign3[-2:] == '12' or sign3[-2:] == '21':
        sign3_order = sign3[-2:]
        sign3 = sign3[:-2]
    else:
        sign3_order = '12'

    if mode == 1:
        # Ordering
        if sign1_order == '21':
            part1_str = f'{num2}{dct[sign1]}{num1}'
        else:
            part1_str = f'{num1}{dct[sign1]}{num2}'

        if sign2_order == '21':
            part2_str = f'{num3}{dct[sign2]}({part1_str})'
        else:
            part2_str =f'({part1_str}){dct[sign2]}{num3}'

        if sign3_order == '21':
            part3_str = f'{num4}{dct[sign3]}({part2_str})'
        else:
            part3_str =f'({part2_str}){dct[sign3]}{num4}'        
    
    if mode == 2:
        if sign1_order == '21':
            part1_str = f'{num2}{dct[sign1]}{num1}'
        else:
            part1_str = f'{num1}{dct[sign1]}{num2}'

        if sign3_order == '21':
            part2_str = f'{num4}{dct[sign3]}{num3}'
        else:
            part2_str = f'{num3}{dct[sign3]}{num4}'
        
        if sign2_order == '21':
            part3_str = f'({part2_str}){dct[sign2]}({part1_str})'
        else:
            part3_str = f'({part1_str}){dct[sign2]}({part2_str})'

    return part3_str

In [499]:
def calculate_all(digits=[2,3,4,5], target=24):
    # 2,3,4,5 -> 2 * (3 + 4 + 5)
    # 1,1,3,9 -> 11 * 3 - 9
    output_str_lst = []
    mode = 0

    cs = list(itertools.combinations(digits,r=2))
    for c in cs:

        remaining_c = two_two_digits(digits, c)
        possible_values_signs = calculate_two_digits(digits=c, allow_combine=True)

        # First case: 2 + 1 + 1
        mode = 1
        for v in possible_values_signs.keys():
            new_digits = [v, remaining_c[0]]
            possible_values_signs2 = calculate_two_digits(digits=new_digits, allow_combine=False)

            for v2 in possible_values_signs2.keys():
                new_digits2 = [v2, remaining_c[1]]
                
                possible_values_signs3 = calculate_two_digits(digits=new_digits2, allow_combine=False)
                if target in possible_values_signs3.keys():
                    formula_str = transform_eq(c[0], possible_values_signs[v], c[1], possible_values_signs2[v2], remaining_c[0], possible_values_signs3[24], remaining_c[1], mode=mode)
                    output_str_lst.append(f'{formula_str} = 24')

        # Second case: 2 + 2
        mode = 2
        possible_values_signs02 = calculate_two_digits(digits=remaining_c, allow_combine=True)
        for v01 in possible_values_signs.keys():
            for v02 in possible_values_signs02.keys():
                new_digits02 = [v01, v02]
                possible_values_signs03 = calculate_two_digits(digits=new_digits02, allow_combine=False)
                if target in possible_values_signs03.keys():
                    formula_str = transform_eq(c[0], possible_values_signs[v01], c[1], possible_values_signs03[24], remaining_c[0], possible_values_signs02[v02], remaining_c[1], mode=mode)
                    output_str_lst.append(f'{formula_str} = 24')

    return output_str_lst

In [517]:
calculate_all(digits=[0,2,3,5])

['(5 ^ 2) - (3 ^ 0) = 24',
 '(5 ^ 0) + (23) = 24',
 '(23) + (5 ^ 0) = 24',
 '(5 ^ 2) - (3 ^ 0) = 24']

In [501]:
score = 0
for a in range(10):
    for b in range(10):
        for c in range(10):
            for d in range(10):
                if len(calculate_all(digits=[a,b,c,d], target=24)) > 0:
                    print(a,b,c,d)
                    score += 1
print('-----------------------')
print(score / 10000)

0 0 2 4
0 0 3 6
0 0 3 8
0 0 4 2
0 0 4 6
0 0 6 3
0 0 6 4
0 0 8 3
0 1 1 4
0 1 2 2
0 1 2 3
0 1 2 4
0 1 2 5
0 1 2 7
0 1 2 8
0 1 3 2
0 1 3 4
0 1 3 5
0 1 3 6
0 1 3 7
0 1 3 8
0 1 3 9
0 1 4 1
0 1 4 2
0 1 4 3
0 1 4 4
0 1 4 5
0 1 4 6
0 1 4 7
0 1 4 8
0 1 5 2
0 1 5 3
0 1 5 4
0 1 5 5
0 1 5 6
0 1 5 9
0 1 6 3
0 1 6 4
0 1 6 5
0 1 6 6
0 1 6 8
0 1 7 2
0 1 7 3
0 1 7 4
0 1 7 7
0 1 7 8
0 1 8 2
0 1 8 3
0 1 8 4
0 1 8 6
0 1 9 3
0 1 9 5
0 2 0 4
0 2 1 2
0 2 1 3
0 2 1 4
0 2 1 5
0 2 1 7
0 2 1 8
0 2 2 1
0 2 2 2
0 2 2 3
0 2 2 4
0 2 2 5
0 2 2 6
0 2 2 8
0 2 3 1
0 2 3 2
0 2 3 3
0 2 3 4
0 2 3 5
0 2 3 6
0 2 3 7
0 2 3 8
0 2 3 9
0 2 4 0
0 2 4 1
0 2 4 2
0 2 4 3
0 2 4 4
0 2 4 5
0 2 4 6
0 2 4 7
0 2 4 8
0 2 4 9
0 2 5 1
0 2 5 2
0 2 5 3
0 2 5 4
0 2 5 5
0 2 5 6
0 2 5 7
0 2 5 8
0 2 5 9
0 2 6 2
0 2 6 3
0 2 6 4
0 2 6 5
0 2 6 6
0 2 6 8
0 2 6 9
0 2 7 1
0 2 7 3
0 2 7 4
0 2 7 5
0 2 7 8
0 2 8 1
0 2 8 2
0 2 8 3
0 2 8 4
0 2 8 5
0 2 8 6
0 2 8 7
0 2 8 8
0 2 8 9
0 2 9 3
0 2 9 4
0 2 9 5
0 2 9 6
0 2 9 8
0 3 0 6
0 3 0 8
0 3 1 2
0 3 1 4
0 3 1 5
